This is the third part of the webscraper.

In [4]:
import sys
sys.path.append('../scripts')  # Add the scripts directory to the system path

# Now you can import your modules from the scripts folder
from auth import get_access_token
from fetch_agency_data import fetch_agency_data

import json

# API client credentials and URL configuration
client_id = 'client_4be73be280ece81767f386e50b2bb7e5'
client_secret = 'secret_3c5eff9a791bd41eeb16feada4a1131b'
scopes = ['api_listings_read']
auth_url = 'https://auth.domain.com.au/v1/connect/token'
api_url_base = 'https://api.domain.com.au/sandbox/v1'

# List of agency IDs
agency_ids = ['32806', '17695', '2795', '18650', '30252', '232', '11765', '12122', '15264', '7797', '17383', '26706', '13833', '29444', '4475', '36174', '4740', '5797', '6242', '7574', '5247', '7261', '4465', '4850', '32569', '30082', '4722', '29117', '17805', '11154', '7920', '18667', '5894', '21571', '7876', '3723', '35065', '33000', '30678', '33307', '24502', '19574', '16586', '18680', '30107', '35358', '35026', '32326', '26073', '31002', '26103']

# Parameters for the GET request
params = {
    'listingStatusFilter': 'liveAndArchived',
    'pageSize': 200  # Fetching 200 listings per page
}

def main():
    # Retrieve access token using the auth module
    access_token = get_access_token(client_id, client_secret, scopes, auth_url)
    
    # Fetch and save listings for each agency
    for agency_id in agency_ids:
        print(f"Fetching data for agency {agency_id}...")
        listings = fetch_agency_data(agency_id, access_token, params, api_url_base)
        
        # Store the data in a JSON file
        with open(f'../data/landing/listings_agency_{agency_id}.json', 'w') as f:
            json.dump(listings, f, indent=2)
        
        print(f"Data for agency {agency_id} saved.\n")

if __name__ == "__main__":
    main()


Fetching data for agency 31835...
Fetched 200 listings from page 1 for agency 31835.
Fetched 144 listings from page 2 for agency 31835.
No more listings found at page 3 for agency 31835.
Data for agency 31835 saved.

Fetching data for agency 28477...
Fetched 200 listings from page 1 for agency 28477.
Fetched 200 listings from page 2 for agency 28477.
Fetched 200 listings from page 3 for agency 28477.
Fetched 75 listings from page 4 for agency 28477.
No more listings found at page 5 for agency 28477.
Data for agency 28477 saved.

Fetching data for agency 19574...
Fetched 200 listings from page 1 for agency 19574.
Fetched 200 listings from page 2 for agency 19574.
Fetched 200 listings from page 3 for agency 19574.
Fetched 200 listings from page 4 for agency 19574.
Fetched 200 listings from page 5 for agency 19574.
Fetched 200 listings from page 6 for agency 19574.
Fetched 200 listings from page 7 for agency 19574.
Fetched 200 listings from page 8 for agency 19574.
Fetched 200 listings fr

In [3]:
import os
import requests
import json
import time

# Client credentials
client_id = 'client_4be73be280ece81767f386e50b2bb7e5'
client_secret = 'secret_3c5eff9a791bd41eeb16feada4a1131b'
scopes = ['api_listings_read']

# Authentication URL
auth_url = 'https://auth.domain.com.au/v1/connect/token'

# Directory where the files are stored
json_dir = '../data/landing/'

# List of agency IDs to scrape (new agencies)
agency_ids = ["5808", "20095", "15964", "20419", "35720"]
#["5005", "14522", "18039", "11825", "6500", "33982", "20864", "11919", "7301", "14503", "32621", "20366", "12612", "35120", "31119", "26113", "21762", "12438", "1638", "11673", "16668", "26896", "26155", "4740", "19325", "32238", "6936", "13411", "25876", "31452", "5808", "10292", "31040", "17234", "11765", "33394", "22721", "7636", "11862", "4714", "19113", "7218", "21537", "32633", "36194", "23154"]
#["32569", "22013", "32851", "30107", "27962", "7728", "20095", "18765", "17337", "15964", "34590", "20419", "4719", "32474", "24322", "3709", "22748", "35557", "32563", "15424", "29733", "22398", "35720", ]

# Parameters for the GET request
params = {
    'listingStatusFilter': 'liveAndArchived',
    'pageSize': 200  # Fetching 200 listings per page
}

def get_access_token():
    response = requests.post(auth_url, data={
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
        'scope': ' '.join(scopes),
        'Content-Type': 'application/x-www-form-urlencoded'
    })
    
    json_response = response.json()
    return json_response['access_token']

def fetch_agency_data(agency_id, access_token):
    headers = {"Authorization": f"Bearer {access_token}"}
    all_listings = []
    
    # Iterate through pages 1 to 300
    for page_number in range(1, 300):
        params['pageNumber'] = page_number
        url_endpoint = f'https://api.domain.com.au/sandbox/v1/agencies/{agency_id}/listings'

        response = requests.get(url_endpoint, headers=headers, params=params)
        
        if response.status_code == 200:
            listings = response.json()
            if not listings:  # If no listings, stop fetching further pages
                print(f"No more listings found at page {page_number} for agency {agency_id}.")
                break
            all_listings.extend(listings)
            print(f"Fetched {len(listings)} listings from page {page_number} for agency {agency_id}.")
        else:
            print(f"Failed to retrieve data for agency {agency_id} on page {page_number}: {response.status_code} - {response.reason}")
            break  # Stop if there's another error

        # Optional: Sleep between requests to avoid rate limiting
        time.sleep(1)
    
    return all_listings

def get_existing_agency_ids(directory):
    """Get a list of agency IDs from the existing JSON files in the directory."""
    existing_files = [f for f in os.listdir(directory) if f.startswith('listings_agency_') and f.endswith('.json')]
    existing_agency_ids = set()

    for file in existing_files:
        try:
            # Extract agency ID, assumes filenames are like: 'listings_agency_<agency_id>_agency_name.json'
            agency_id = file.split('_')[2]
            existing_agency_ids.add(agency_id)
        except IndexError:
            # Handle unexpected file names
            print(f"Could not extract agency ID from {file}")
    
    return existing_agency_ids

def main():
    access_token = get_access_token()
    
    # Get the set of existing agency IDs
    existing_agency_ids = get_existing_agency_ids(json_dir)

    # Filter out agency IDs that already have a file
    new_agency_ids = [agency_id for agency_id in agency_ids if agency_id not in existing_agency_ids]
    
    if not new_agency_ids:
        print("All agencies have already been scraped.")
        return

    for agency_id in new_agency_ids:
        print(f"Fetching data for agency {agency_id}...")
        listings = fetch_agency_data(agency_id, access_token)
        
        # Store the data (e.g., save to a JSON file)
        with open(f'{json_dir}new/listings_agency_{agency_id}.json', 'w') as f:
            json.dump(listings, f, indent=2)
        
        print(f"Data for agency {agency_id} saved.\n")

if __name__ == "__main__":
    main()


Fetching data for agency 5808...


KeyboardInterrupt: 